In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from geopy.distance import geodesic
from sklearn.cluster import KMeans

# Calculate distance from home function
def calculate_distance(row):
    home_location = (row['lat'], row['long'])
    merch_location = (row['merch_lat'], row['merch_long'])
    return geodesic(home_location, merch_location).miles

# Function to calculate distance between two points
def calculate_distance2(row1, row2):
    point1 = (row1['lat'], row1['long'])
    point2 = (row2['lat'], row2['long'])
    return geodesic(point1, point2).miles

def calculate_similarity_score(amount, fraud_mean, fraud_std, normal_mean, normal_std):
    # Calculate Z-scores for fraud and normal
    z_score_fraud = abs((amount - fraud_mean) / fraud_std)
    z_score_normal = abs((amount - normal_mean) / normal_std)
    
    # Invert the Z-scores to get similarity scores
    fraud_similarity = 1 / (1 + z_score_fraud)
    normal_similarity = 1 / (1 + z_score_normal)
    
    return fraud_similarity, normal_similarity

def process(df):
    # Add new features
    # Rearrange the rows
    df['original_order'] = range(df.shape[0])

    df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'], format='%d/%m/%Y %H:%M')
    df['dob'] = pd.to_datetime(df['dob'], format='%d/%m/%Y')

    df.sort_values(by=['cc_num', 'trans_date_trans_time'], inplace=True)
    
    # Calculate the time difference between transactions
    df['Time_Delta'] = df.groupby('cc_num')['trans_date_trans_time'].diff().dt.total_seconds() / 60.0  # Time delta in minutes
    df['Time_Delta'] = df['Time_Delta'].fillna(value=0)
    
    # Shift the latitude and longitude to get the previous transaction's location
    df['prev_lat'] = df.groupby('cc_num')['merch_lat'].shift(1)
    df['prev_long'] = df.groupby('cc_num')['merch_long'].shift(1)

    # Calculate the distance to the previous transaction
    df['distance_to_prev'] = df.apply(
        lambda row: calculate_distance2(
            {'lat': row['merch_lat'], 'long': row['merch_long']},
            {'lat': row['prev_lat'], 'long': row['prev_long']}
        ) if not pd.isnull(row['prev_lat']) else None,
        axis=1
    )
    
    df['distance_to_prev'] = df['distance_to_prev'].fillna(value=0)

    # Calculate location consistency as the inverse of the average distance to previous transactions (higher value means more consistency)
    df['location_consistency'] = 100 / df.groupby('cc_num')['distance_to_prev'].transform('mean')

    # Time-based features
    df['hour'] = df['trans_date_trans_time'].dt.hour
    df['day_of_week'] = df['trans_date_trans_time'].dt.dayofweek
    df['month'] = df['trans_date_trans_time'].dt.month
    df['day_of_month'] = df['trans_date_trans_time'].dt.day
   
    # Age of the account holder
    df['age'] = (df['trans_date_trans_time'] - df['dob']).dt.days // 365

    df['dist_to_home'] = df.apply(calculate_distance, axis=1)

    # Group by category and calculate the mean and standard deviation of transaction amounts
    category_stats = df.groupby('category')['amt'].agg(['mean', 'std']).reset_index()
    # Merge these stats back into the main dataframe
    df = df.merge(category_stats, on='category', how='left')
    # Calculate z-score for each transaction amount within its category
    df['amt_anomaly_score_cat'] = ((df['amt'] - df['mean']) / df['std'])
    df.drop(columns=['mean', 'std'], inplace=True)
    # Group by merchant and calculate the mean and standard deviation of transaction amounts
    merchant_stats = df.groupby('merchant')['amt'].agg(['mean', 'std']).reset_index()
    # Merge these stats back into the main dataframe
    df = df.merge(merchant_stats, on='merchant', how='left')
    # Calculate z-score for each transaction amount within its merchant
    df['amt_anomaly_score_merch'] = ((df['amt'] - df['mean']) / df['std'])
    df.drop(columns=['mean', 'std'], inplace=True)

    # Calculate the historical average transaction amount for each user
    avg_amt_per_user = df.groupby('cc_num')['amt'].transform('mean').rename('avg_amt_per_user')

    # Append this feature to the dataset
    df['amt_relative_avg'] = (abs(df['amt'] - avg_amt_per_user) / avg_amt_per_user)

    kmeans = KMeans(n_clusters=12, random_state=42)

    # Create a new column for the cluster labels
    df['city_pop_cluster'] = kmeans.fit_predict(df[['city_pop']])

    df.drop(columns=['trans_date_trans_time', 'lat', 'long', 'merch_lat', 'merch_long'], inplace=True)
    df.drop(columns=['prev_lat', 'prev_long'], inplace=True)

    # Identify categorical columns to encode
    categorical_cols = ['merchant', 'category', 'gender', 'city', 'state', 'job']

    mappings = {}

    label_encoder = LabelEncoder()
    for col in categorical_cols:
        df[col] = label_encoder.fit_transform(df[col])
        mappings[col] = {label: index for index, label in enumerate(label_encoder.classes_)}


    # Calculate the fraud rate by category
    fraud_rate_by_category = df.groupby('category')['is_fraud'].mean().reset_index()
    fraud_rate_by_category.rename(columns={'is_fraud': 'fraud_rate_cat'}, inplace=True)

    # Merge the fraud rate back into the main DataFrame
    df = pd.merge(df, fraud_rate_by_category[['category', 'fraud_rate_cat']], on='category', how='left')

    # Calculate the fraud rate by merchant
    fraud_rate_by_merchant = df.groupby('merchant')['is_fraud'].mean().reset_index()
    fraud_rate_by_merchant.rename(columns={'is_fraud': 'fraud_rate_merch'}, inplace=True)

    # Merge the fraud rate back into the main DataFrame
    df = pd.merge(df, fraud_rate_by_merchant[['merchant', 'fraud_rate_merch']], on='merchant', how='left')

    # Separate the transactions
    fraud_trans = df[df['is_fraud'] == 1]['amt']
    normal_trans = df[df['is_fraud'] == 0]['amt']

    # Calculate statistics
    fraud_mean, fraud_std = fraud_trans.mean(), fraud_trans.std()
    normal_mean, normal_std = normal_trans.mean(), normal_trans.std()

    v_calculate_similarity_score = np.vectorize(calculate_similarity_score)

    # Apply the function
    df['fraud_similarity'], df['normal_similarity'] = v_calculate_similarity_score(
        df['amt'],
        fraud_mean, fraud_std,
        normal_mean, normal_std
    )

    # Sort the dataset back to its original order
    df.sort_values(by='original_order', inplace=True)
    df.drop(columns='original_order', inplace=True)
    df.drop(columns='cc_num', inplace=True)

    return df, mappings

trainingSet = pd.read_csv("./data/train.csv")
submissionSet = pd.read_csv("./data/test.csv")
train_processed, cat_map = process(trainingSet)
train_processed.drop(columns=['first', 'last', 'street', 'dob', 'zip', 'trans_num', 'unix_time'], inplace=True)

# Merge on Id so that the test set can have feature columns as well
test_df = pd.merge(train_processed, submissionSet, left_on='Id', right_on='Id')
test_df = test_df.drop(columns=['is_fraud_x'])
test_df = test_df.rename(columns={'is_fraud_y': 'is_fraud'})

# The training set is where the score is not null
train_df = train_processed[train_processed['is_fraud'].notnull()]

# Save the datasets with the new features for easy access later
test_df.to_csv("./data/final_processed_test.csv", index=False)
train_df.to_csv("./data/final_processed_train.csv", index=False)

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Assuming 'train_df' includes both features and the target ('is_fraud')
X = train_df.drop(['is_fraud', 'Id', 'city_pop_cluster'], axis=1)
y = train_df['is_fraud']

num_cols = ['amt', 'Time_Delta', 'distance_to_prev', 'location_consistency', 'dist_to_home', 'amt_anomaly_score_cat', 'amt_anomaly_score_merch', 'amt_relative_avg', 'fraud_rate_cat', 'fraud_rate_merch', 'fraud_similarity', 'normal_similarity']

scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])
test_df[num_cols] = scaler.transform(test_df[num_cols])

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [3]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, make_scorer
# Initialize the XGBClassifier
xgb_clf = XGBClassifier(colsample_bytree=0.9, learning_rate=0.1, max_depth=5, n_estimators=900, use_label_encoder=False, eval_metric='logloss')

xgb_clf.fit(X_train, y_train)

y_pred_xgb = xgb_clf.predict(X_val)
print(f1_score(y_val, y_pred_xgb))
print(classification_report(y_val, y_pred_xgb))

0.9085714285714286
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     96876
         1.0       0.98      0.85      0.91       375

    accuracy                           1.00     97251
   macro avg       0.99      0.92      0.95     97251
weighted avg       1.00      1.00      1.00     97251



In [4]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, make_scorer
# Initialize the XGBClassifier
xgb_clf2 = XGBClassifier(colsample_bytree=0.9, learning_rate=0.15, max_depth=None, n_estimators=900, use_label_encoder=False, eval_metric='logloss')

xgb_clf2.fit(X_train, y_train)

y_pred_xgb2 = xgb_clf2.predict(X_val)
print(f1_score(y_val, y_pred_xgb2))
print(classification_report(y_val, y_pred_xgb2))

0.9085714285714286
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     96876
         1.0       0.98      0.85      0.91       375

    accuracy                           1.00     97251
   macro avg       0.99      0.92      0.95     97251
weighted avg       1.00      1.00      1.00     97251



In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
import pickle
from xgboost import XGBClassifier
import numpy as np
from sklearn.ensemble import BaggingClassifier
# Create a VotingClassifier
from sklearn.ensemble import VotingClassifier

# Initialize the Bagging Classifier
bagging_clf = BaggingClassifier(estimator=xgb_clf, n_estimators=100, random_state=42, n_jobs=-1, verbose=3)

bagging_clf.fit(X_train, y_train)
y_pred_bag = bagging_clf.predict(X_val)
print(f1_score(y_val, y_pred_bag))

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this para

[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  3.7min remaining:  8.6min
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:  3.7min remaining:  1.6min
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  3.7min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:   15.3s remaining:   35.8s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:   15.7s remaining:    6.7s


0.8694362017804155


[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   16.3s finished


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
import pickle
from xgboost import XGBClassifier
import numpy as np
from sklearn.ensemble import BaggingClassifier
# Create a VotingClassifier
from sklearn.ensemble import VotingClassifier

rf1 = RandomForestClassifier(class_weight='balanced_subsample', n_estimators=300, max_depth=None, min_samples_split=10, min_samples_leaf=4, criterion='entropy', n_jobs=-1, random_state=42)

rf1.fit(X_train, y_train)

y_pred_rf1 = rf1.predict(X_val)
print(f1_score(y_val, y_pred_rf1))
print(classification_report(y_val, y_pred_rf1))

0.8115107913669065
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     96876
         1.0       0.88      0.75      0.81       375

    accuracy                           1.00     97251
   macro avg       0.94      0.88      0.91     97251
weighted avg       1.00      1.00      1.00     97251



In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
import pickle
from xgboost import XGBClassifier
import numpy as np
from sklearn.ensemble import BaggingClassifier
# Create a VotingClassifier
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[('rf1', rf1), ('bag', bagging_clf), ('xgb', xgb_clf), ('xgb2', xgb_clf2)],
    voting='soft',
    n_jobs=-1
)

# Train the ensemble model
vot_clf = voting_clf.fit(X_train, y_train)

y_pred_vote = vot_clf.predict(X_val)
f1_score_vote = f1_score(y_val, y_pred_vote)
print(f1_score_vote)


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...Building estimator 1 of 10 for this parallel run (total 100)...

Building estimator 1 of 10 for this parallel run (total 100)...Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...

Building estimator 1 of 10 for this parallel run (total 100)...Building estimator 1 of 10 for this parallel run (total 100)...

Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this para

[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  3.9min remaining:  9.2min
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:  4.0min remaining:  1.7min
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  4.0min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:   15.8s remaining:   36.9s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:   16.3s remaining:    7.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   16.4s finished


0.888243831640058


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
import pickle
from xgboost import XGBClassifier
import numpy as np
from sklearn.ensemble import BaggingClassifier
# Create a VotingClassifier
from sklearn.ensemble import VotingClassifier

voting_clf2 = VotingClassifier(
    estimators=[('xgb', xgb_clf), ('bag', bagging_clf), ('rf1', rf1)],
    voting='soft',
    n_jobs=-1
)

# Train the ensemble model
vot_clf2 = voting_clf2.fit(X_train, y_train)

y_pred_vote2 = vot_clf2.predict(X_val)
f1_score_vote2 = f1_score(y_val, y_pred_vote2)
print(f1_score_vote2)


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...Building estimator 1 of 10 for this parallel run (total 100)...

Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 1 of 10 for this parallel run (total 100)...

Building estimator 1 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this parallel run (total 100)...
Building estimator 2 of 10 for this para

[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  3.9min remaining:  9.1min
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:  3.9min remaining:  1.7min
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  3.9min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:   15.1s remaining:   35.2s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:   15.9s remaining:    6.8s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   16.1s finished


0.8859649122807017


In [17]:
# Create Kaggle Submission
pred = test_df.drop(['is_fraud', 'city_pop_cluster', 'Id'], axis=1)
pred2 = test_df.drop(['is_fraud'], axis=1)

pred2['is_fraud'] = vot_clf.predict(pred)
pred2.is_fraud = pred2.is_fraud.astype(int)
submission = pred2[['Id', 'is_fraud']]
submission.to_csv("./data/competition_submission_final.csv", index=False)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:   10.9s remaining:   25.4s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:   11.4s remaining:    4.9s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   11.7s finished


In [15]:
# Create Kaggle Submission
pred3 = test_df.drop(['is_fraud'], axis=1)

pred3['is_fraud'] = vot_clf2.predict(pred)
pred3.is_fraud = pred3.is_fraud.astype(int)
submission2 = pred3[['Id', 'is_fraud']]
submission2.to_csv("./data/competition_submission_final2.csv", index=False)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:   11.7s remaining:   27.4s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:   12.0s remaining:    5.2s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   12.3s finished


In [18]:
import pickle

with open('competitioin_model_final.obj', 'wb') as f:
    pickle.dump(vot_clf, f)

with open('competitioin_model_final2.obj', 'wb') as f:
    pickle.dump(vot_clf2, f)

In [20]:
# Create Kaggle Submission
pred4 = test_df.drop(['is_fraud'], axis=1)

pred4['is_fraud'] = xgb_clf.predict(pred)
pred4.is_fraud = pred4.is_fraud.astype(int)
submission2 = pred4[['Id', 'is_fraud']]
submission2.to_csv("./data/competition_submission_final3.csv", index=False)